## Transition from Volatile RAM Checkpoints to Persistent SQLite Checkpoints in LangGraph



---

# 1. Architecture

## 1.1 System Components

* LangGraph runtime engine (checkpointing + node scheduling)
* SQLite persistence checkpointing using `SqliteSaver`
* Streamlit UI front-end
* OpenAI-compatible chat LLM backend
* Local storage filesystem for `chatbot.db`

## 1.2 Dataflow

User input → Streamlit UI → LangGraph → Chat Node → LLM Invocation → Response → Checkpoint persisted to SQLite → Resume Execution

## 1.3 Trust Boundaries

* UI boundary: User input enters application (untrusted)
* LLM boundary: Prompts and responses (semi-trusted)
* Database boundary: Local SQLite store (trusted)
* OS boundary: Local filesystem (trusted)

Input validation performed at the UI boundary. LLM outputs are treated as untrusted content and sanitized for rendering.

## 1.4 Diagram (Mermaid)

```mermaid
flowchart TD
    UI[Streamlit UI] -->|text input| LG[LangGraph Engine]
    LG --> DB[(SQLite Checkpoints)]
    LG --> LLM[Chat LLM]
    LLM --> LG
    LG --> UI
```

---

# 2. Implementation

## 2.1 Dependencies (Pinned)

```
python==3.11.8
langgraph-checkpoint-sqlite==0.0.3
langchain-openai==0.2.7
streamlit==1.36.0
sqlite3 (stdlib)
uuid (stdlib)
```

---

## 2.2 Backend: Persistent SQLite Checkpoints

```python
# backend.py

import sqlite3
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage
from dotenv import load_dotenv

load_dotenv()

class ChatState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

def chat_node(state: ChatState) -> dict:
    messages = state["messages"]
    if not isinstance(messages, list):
        raise TypeError("messages must be a list")
    llm = ChatOpenAI()
    response = llm.invoke(messages)
    return {"messages": [response]}

conn = sqlite3.connect("chatbot.db", check_same_thread=False)
checkpointer = SqliteSaver(conn)

graph = StateGraph(ChatState)
graph.add_node("chat_node", chat_node)
graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)
chatbot = graph.compile(checkpointer=checkpointer)

def retrieve_all_threads() -> list:
    result = set()
    for cp in checkpointer.list(None):
        result.add(cp.config["configurable"]["thread_id"])
    return list(result)
```

---

## 2.3 Frontend (Streamlit Integration)

```python
# app.py

import streamlit as st
import uuid
from typing import List
from langchain_core.messages import HumanMessage
from backend import chatbot, retrieve_all_threads

def generate_thread_id() -> str:
    return str(uuid.uuid4())

def validate_thread_id(value: str) -> str:
    if not isinstance(value, str):
        raise ValueError("thread_id must be str")
    return value

def load_conversation(thread_id: str) -> List[dict]:
    state = chatbot.get_state(config={"configurable": {"thread_id": thread_id}})
    msgs = state.values.get("messages", [])
    result = []
    for msg in msgs:
        role = "user" if isinstance(msg, HumanMessage) else "assistant"
        result.append({"role": role, "content": msg.content})
    return result

if "thread_id" not in st.session_state:
    st.session_state["thread_id"] = generate_thread_id()
if "chat_threads" not in st.session_state:
    st.session_state["chat_threads"] = retrieve_all_threads()
if "message_history" not in st.session_state:
    st.session_state["message_history"] = []

st.sidebar.title("LangGraph Chatbot")

if st.sidebar.button("New Chat"):
    new_id = generate_thread_id()
    st.session_state["thread_id"] = new_id
    st.session_state["chat_threads"].append(new_id)
    st.session_state["message_history"] = []

st.sidebar.header("Conversations")

for tid in st.session_state["chat_threads"][::-1]:
    if st.sidebar.button(tid):
        validate_thread_id(tid)
        st.session_state["thread_id"] = tid
        st.session_state["message_history"] = load_conversation(tid)

for m in st.session_state["message_history"]:
    with st.chat_message(m["role"]):
        st.text(m["content"])

user_input = st.chat_input("Type here")

if user_input:
    st.session_state["message_history"].append({"role": "user", "content": user_input})
    with st.chat_message("user"):
        st.text(user_input)

    config = {
        "configurable": {"thread_id": st.session_state["thread_id"]},
        "metadata": {"thread_id": st.session_state["thread_id"]},
        "run_name": "chat_turn",
    }

    with st.chat_message("assistant"):
        output = st.write_stream(
            chunk.content for chunk, _meta in chatbot.stream(
                {"messages": [HumanMessage(content=user_input)]},
                config=config,
                stream_mode="messages"
            )
        )
    st.session_state["message_history"].append({"role": "assistant", "content": output})
```




---

## 1. Difference between a checkpoint and a message buffer

A checkpoint is a persisted snapshot of graph execution state (messages, node outputs, config, execution metadata). It enables full resumption of execution at deterministic boundaries. A message buffer is a temporary in-memory collection of user/agent messages used to construct the prompt and produce the next model invocation. The checkpoint survives process restarts; the buffer does not.

---

## 2. How LangGraph schedules nodes and resolves state

LangGraph executes via state machines. Nodes define transformation functions over a typed state dictionary. Execution proceeds through edges defined between nodes until termination conditions are met. Each node invocation receives immutable state and returns a deterministic diff to be merged. This produces a linear or branching state evolution depending on graph topology. Scheduling completes when the END marker is reached or when multiple branches converge. Checkpoints capture state after “supersteps.”

---

## 3. Durability guarantees of SQLite vs PostgreSQL

SQLite uses a single-file ACID-compliant storage engine with transactional semantics and journaling. Durability is local to the filesystem with no built-in replication. PostgreSQL implements WAL (write-ahead-logging) with background flush, crash recovery, multi-process concurrency, replication (streaming, logical, synchronous), multi-region failover, and hot backups. PostgreSQL supports stronger durability guarantees in distributed deployments.

---

## 4. Why `check_same_thread=False` is required

The SQLite Python driver restricts DB operations to the thread that created the connection. LangGraph may handle different conversation threads via multiple UI or scheduler threads. Disabling the thread check allows the same connection object to be shared across threads. Without it, calls during checkpoint writes raise `ProgrammingError`.

---

## 5. Thread isolation in multi-agent chat systems

Isolation is achieved by namespacing memory and state by a thread identifier. Each thread’s checkpoint log forms a separate execution lineage so concurrent interactions do not pollute each other’s context. Isolation can be at the key level (thread_id), row level, schema level, or database level.

---

## 6. ACID and how SQLite implements it

ACID = Atomicity, Consistency, Isolation, Durability. SQLite implements it using journaling (rollback journal or WAL), coarse-grained write locks, and atomic filesystem operations. On crash, uncommitted writes are discarded via journal replay. Durability is guaranteed when synchronous=FULL and fsync completes before commit returns.

---

## 7. Recovery from partial checkpoint writes

During a partial write or crash, checkpointing replays the journal. Committed transactions in WAL are applied during recovery. Uncommitted frames are discarded. For rollback journal mode, the original page content is restored from the rollback journal, preserving pre-transaction state.

---

## 8. WAL vs rollback journal tradeoff

Rollback journal copies original pages before writing new ones. WAL appends changes sequentially to a WAL file and checkpoints back to the main DB later. WAL improves concurrency (multiple readers + single writer), write throughput, and reduces page copying. Rollback journal has simpler guarantees but lower concurrency.

---

## 9. Schema-less JSON storage for message envelopes

Message content can be encoded as JSON text within a TEXT or BLOB column. This avoids migration churn as envelope shape evolves. Retrieval is done via application-level parsing. PostgreSQL would additionally offer JSONB indexing, which SQLite lacks natively.

---

## 10. Migration strategy SQLite → PostgreSQL

Typical pipeline:

1. schema creation in PostgreSQL
2. extraction from SQLite
3. transformation of JSON envelopes
4. load into PostgreSQL
5. update application to point at new DSN
6. cutover with freeze period or dual-write
7. decommission SQLite after validation

Node checkpoint format does not need to change assuming envelopes remain JSON-compatible.

---

